## 8강_ 영화 추천 시스템

이번 강의에서는, Kaggle의 'Movielens'dataset을 이용하여,

영화 추천 시스템을 만들어보도록 하겠습니다!!

우선 두 파일을 다운받으세요 - 아래 링크에 접속하여서, 

https://www.kaggle.com/sengzhaotoo/movielens-small

좌측의 movie.csv, ratings.csv 파일을 다운받으신 후, 

이 .ipynb파일과 같은 경로에 저장하세요!!



pip 명령어로 numpy, pandas, sklearn을 설치해 줍니다!!
예시) !pip install numpy pandas sklearn

ratings.csv 파일은 각 사용자가 영화에 몇 점을 주었는지(총평점 5점 중 몇 점을 받았는가)를 나타내주는 파일입니다.

해당 파일을 파이썬 콘솔로 불러와 살펴보도록 할까요?

In [19]:
import numpy as np # linear algebra
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
ratings=pd.read_csv('ratings.csv')
# ratings.head(15)
ratings.tail(15)

,userId,movieId,rating,timestamp
99989,671,5445,4.5,1064891627
99990,671,5464,3.0,1064891549
99991,671,5669,4.0,1063502711
99992,671,5816,4.0,1065111963
99993,671,5902,3.5,1064245507
99994,671,5952,5.0,1063502716
99995,671,5989,4.0,1064890625
99996,671,5991,4.5,1064245387
99997,671,5995,4.0,1066793014
99998,671,6212,2.5,1065149436


movies.csv 파일은 영화의 목록 파일입니다.

장르도 같이 포함하고 있어요!!(복수 장르)

불러와서 내용을 살펴보도록 하겠습니다!!

In [25]:
movies=pd.read_csv('movies.csv')
movies.head(15)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


이제 두 Dataframe을 합쳐서(merge, 병합해서) 나타내보도록 하겠습니다!!

In [23]:
movie_ratings = pd.merge(movies, ratings)
movie_ratings.head(15)

,movieId,title,genres,userId,rating,timestamp
99989,160565,The Purge: Election Year (2016),Action|Horror|Sci-Fi,15,2.0,1469330266
99990,160567,Mike & Dave Need Wedding Dates (2016),Comedy,15,4.0,1469330242
99991,160590,Survive and Advance (2013),(no genres listed),56,5.0,1467095789
99992,160656,Tallulah (2016),Drama,547,3.5,1471168466
99993,160718,Piper (2016),Animation,547,4.0,1469713151
99994,161084,My Friend Rockefeller (2015),Documentary,547,2.5,1468248331
99995,161155,Sunspring (2016),Sci-Fi,15,0.5,1469330307
99996,161594,Kingsglaive: Final Fantasy XV (2016),Action|Adventure|Animation|Drama|Fantasy|Sci-Fi,73,3.0,1474255459
99997,161830,Body (2015),Drama|Horror|Thriller,624,1.0,1472493010
99998,161918,Sharknado 4: The 4th Awakens (2016),Action|Adventure|Horror|Sci-Fi,624,1.5,1472929873


## 추천 시스템 구성

아래는 추천 시스템을 구성하기 위해, userid 와 영화 별 평점을 도표로 나타낸 것입니다!!

가로축은 영화의 id,

세로축은 평점을 준 user id를 나타내고 있어요!!


In [20]:
# 테이블을 pivot하여 가로축에는 영화 id (671번까지)
# 세로축에는 user id (100003번까지)를 위치시켜,
# matrix를 만들어 줍니다!! (가로세로 그물망)
ratings_matrix = ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
# 빈 칸에 0을 삽입함으로서 에러가 나지 않도록 합니다!!
ratings_matrix.fillna( 0, inplace = True ) 
#상위 15개 행만 출력합니다!!
ratings_matrix.head(15)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,4.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


아래는 추천 시스템을 구성하기 위해, cosine_similarty를 사용한 것입니다!!

즉 같은 사용자가 비슷한 추천 점수를 준 영화(아이템)이라면, 유사도가 높다고 평가하는 것입니다!!

Reference: https://lsjsj92.tistory.com/563

In [11]:
movie_similarity=cosine_similarity(ratings_matrix)
np.fill_diagonal( movie_similarity, 0 ) 
movie_similarity

array([[0.        , 0.39451145, 0.30651588, ..., 0.        , 0.        ,
        0.05582876],
       [0.39451145, 0.        , 0.21749153, ..., 0.        , 0.        ,
        0.        ],
       [0.30651588, 0.21749153, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.05582876, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [22]:
ratings_matrix = pd.DataFrame( movie_similarity )
ratings_matrix.head(15)
# 9124 x 9124 의 matrix를 만들어, 각 영화들끼리의 유사도를 나타내 줍니다!!

,0,1,2,3,4,5,6,7,8,9,...,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
0,0.000000,0.394511,0.306516,0.133614,0.245102,0.377086,0.278629,0.063031,0.117499,0.310689,...,0.055829,0.031902,0.079755,0.079755,0.079755,0.079755,0.079755,0.0,0.0,0.055829
1,0.394511,0.000000,0.217492,0.164651,0.278476,0.222003,0.207299,0.223524,0.113669,0.418124,...,0.000000,0.055038,0.068797,0.082557,0.082557,0.137594,0.068797,0.0,0.0,0.000000
2,0.306516,0.217492,0.000000,0.177012,0.370732,0.247499,0.435648,0.127574,0.306717,0.191255,...,0.000000,0.000000,0.000000,0.116226,0.116226,0.000000,0.000000,0.0,0.0,0.000000
3,0.133614,0.164651,0.177012,0.000000,0.179556,0.072518,0.184626,0.501513,0.254630,0.111447,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,0.245102,0.278476,0.370732,0.179556,0.000000,0.272645,0.388476,0.194113,0.367941,0.246846,...,0.000000,0.176845,0.000000,0.117897,0.117897,0.000000,0.000000,0.0,0.0,0.000000
5,0.377086,0.222003,0.247499,0.072518,0.272645,0.000000,0.278855,0.097561,0.248155,0.307948,...,0.061724,0.098758,0.111103,0.000000,0.000000,0.000000,0.111103,0.0,0.0,0.061724
6,0.278629,0.207299,0.435648,0.184626,0.388476,0.278855,0.000000,0.196091,0.349827,0.177425,...,0.079399,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.079399
7,0.063031,0.223524,0.127574,0.501513,0.194113,0.097561,0.196091,0.000000,0.264477,0.042169,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
8,0.117499,0.113669,0.306717,0.254630,0.367941,0.248155,0.349827,0.264477,0.000000,0.130475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9,0.310689,0.418124,0.191255,0.111447,0.246846,0.307948,0.177425,0.042169,0.130475,0.000000,...,0.000000,0.076835,0.076835,0.102446,0.102446,0.000000,0.076835,0.0,0.0,0.000000


In [13]:
try:
    #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
    user_inp="Jumanji (1995)"
    inp=movies[movies['title']==user_inp].index.tolist()
    inp=inp[0]
    
    movies['similarity'] = ratings_matrix.iloc[inp]
    movies.head(5)
    
except:
    print("Sorry, the movie is not in the database!")
    
print("Recommended movies based on your choice of ",user_inp ,": \n", movies.sort_values( ["similarity"], ascending = False )[1:10])

Recommended movies based on your choice of  Jumanji (1995) : 
      movieId                              title  \
328      364              Lion King, The (1994)   
283      317           Santa Clause, The (1994)   
331      367                   Mask, The (1994)   
527      595        Beauty and the Beast (1991)   
521      588                     Aladdin (1992)   
309      344  Ace Ventura: Pet Detective (1994)   
520      587                       Ghost (1990)   
427      480               Jurassic Park (1993)   
341      377                       Speed (1994)   

                                              genres  similarity  
328  Adventure|Animation|Children|Drama|Musical|IMAX    0.530357  
283                             Comedy|Drama|Fantasy    0.505831  
331                      Action|Comedy|Crime|Fantasy    0.494605  
527  Animation|Children|Fantasy|Musical|Romance|IMAX    0.494124  
521      Adventure|Animation|Children|Comedy|Musical    0.493995  
309                     